# **Importing Libraries**

In [11]:
import csv
import pandas as pd
import numpy as np
import re
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.models import Model

# **Creating the CSV file**

In [10]:
# Define a list of dictionaries containing metadata, COBOL code, and its Java translation
cobol_java_pairs = [
    {
        "ID": 1,
        "Domain": "General",
        "Purpose": "HelloWorld",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. HelloWorld.
                        PROCEDURE DIVISION.
                        P1.
                            DISPLAY "Hello, World!".
                            STOP RUN.""",
        "Java Translation": """public class HelloWorld {
                                  public static void main(String[] args) {
                                      System.out.println("Hello, World!");
                                  }
                              }"""
    },
    {
        "ID": 2,
        "Domain": "Arithmetic",
        "Purpose": "AddNumbers",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. AddNumbers.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 A PIC 9(2) VALUE 10.
                        01 B PIC 9(2) VALUE 20.
                        01 SUM PIC 9(4).
                        PROCEDURE DIVISION.
                        P1.
                            COMPUTE SUM = A + B.
                            DISPLAY "SUM=", SUM.
                            STOP RUN.""",
        "Java Translation": """public class AddNumbers {
                                  public static void main(String[] args) {
                                      int A = 10;
                                      int B = 20;
                                      int SUM;
                                      SUM = A + B;
                                      System.out.println("SUM=" + SUM);
                                  }
                              }"""
    },
    {
        "ID": 3,
        "Domain": "Arithmetic",
        "Purpose": "LargestNumber",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. LargestNumber.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 A PIC 9(2) VALUE 10.
                        01 B PIC 9(2) VALUE 20.
                        01 MAX PIC 9(2).
                        PROCEDURE DIVISION.
                        P1.
                            IF A > B THEN
                                MOVE A TO MAX
                            ELSE
                                MOVE B TO MAX
                            END-IF.
                            DISPLAY "MAX=", MAX.
                            STOP RUN.""",
        "Java Translation": """public class LargestNumber {
                                  public static void main(String[] args) {
                                      int A = 10;
                                      int B = 20;
                                      int MAX;
                                      if (A > B) {
                                          MAX = A;
                                      } else {
                                          MAX = B;
                                      }
                                      System.out.println("MAX=" + MAX);
                                  }
                              }"""
    },
    {
        "ID": 4,
        "Domain": "Mathematical",
        "Purpose": "Factorial",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. Factorial.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9(2) VALUE 5.
                        01 FACTORIAL PIC 9(10).
                        PROCEDURE DIVISION.
                        P1.
                            COMPUTE FACTORIAL = 1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > N
                                COMPUTE FACTORIAL = FACTORIAL * I
                            END-PERFORM.
                            DISPLAY "FACTORIAL=", FACTORIAL.
                            STOP RUN.""",
        "Java Translation": """public class Factorial {
                                  public static void main(String[] args) {
                                      int N = 5;
                                      long FACTORIAL = 1;
                                      for (int I = 1; I <= N; I++) {
                                          FACTORIAL *= I;
                                      }
                                      System.out.println("FACTORIAL=" + FACTORIAL);
                                  }
                              }"""
    },
    {
        "ID": 5,
        "Domain": "Mathematical",
        "Purpose": "FibonacciSeries",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. FibonacciSeries.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 N PIC 9(2) VALUE 10.
                        01 FIB1 PIC 9(10) VALUE 0.
                        01 FIB2 PIC 9(10) VALUE 1.
                        01 NEXTFIB PIC 9(10).
                        PROCEDURE DIVISION.
                        P1.
                            DISPLAY FIB1.
                            DISPLAY FIB2.
                            PERFORM UNTIL N <= 2
                                COMPUTE NEXTFIB = FIB1 + FIB2
                                DISPLAY NEXTFIB
                                COMPUTE FIB1 = FIB2
                                COMPUTE FIB2 = NEXTFIB
                                COMPUTE N = N - 1
                            END-PERFORM.
                            STOP RUN.""",
        "Java Translation": """public class FibonacciSeries {
                                  public static void main(String[] args) {
                                      int N = 10;
                                      int FIB1 = 0;
                                      int FIB2 = 1;
                                      int NEXTFIB;
                                      System.out.println(FIB1);
                                      System.out.println(FIB2);
                                      while (N > 2) {
                                          NEXTFIB = FIB1 + FIB2;
                                          System.out.println(NEXTFIB);
                                          FIB1 = FIB2;
                                          FIB2 = NEXTFIB;
                                          N--;
                                      }
                                  }
                              }"""
    },
    {
        "ID": 6,
        "Domain": "File Handling",
        "Purpose": "ReadFile",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. ReadFile.
                        ENVIRONMENT DIVISION.
                        INPUT-OUTPUT SECTION.
                        FILE-CONTROL.
                            SELECT InputFile ASSIGN TO 'input.txt'.
                        DATA DIVISION.
                        FILE SECTION.
                        FD InputFile.
                        01 InputRecord PIC X(50).
                        WORKING-STORAGE SECTION.
                        01 EOF-SWITCH PIC X VALUE 'N'.
                        PROCEDURE DIVISION.
                        P1.
                            OPEN INPUT InputFile
                            READ InputFile INTO InputRecord
                            AT END MOVE 'Y' TO EOF-SWITCH
                            END-READ
                            PERFORM UNTIL EOF-SWITCH = 'Y'
                                DISPLAY InputRecord
                                READ InputFile INTO InputRecord
                                AT END MOVE 'Y' TO EOF-SWITCH
                                END-READ
                            END-PERFORM
                            CLOSE InputFile
                            STOP RUN.""",
        "Java Translation": """import java.io.BufferedReader;
                                import java.io.FileReader;
                                import java.io.IOException;

                                public class ReadFile {
                                    public static void main(String[] args) {
                                        try {
                                            BufferedReader reader = new BufferedReader(new FileReader("input.txt"));
                                            String line;
                                            while ((line = reader.readLine()) != null) {
                                                System.out.println(line);
                                            }
                                            reader.close();
                                        } catch (IOException e) {
                                            e.printStackTrace();
                                        }
                                    }
                                }"""
    },
    {
        "ID": 7,
        "Domain": "String Manipulation",
        "Purpose": "ReverseString",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. ReverseString.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 InputString PIC X(50) VALUE 'Hello, World!'.
                        01 OutputString PIC X(50).
                        01 I PIC 9(3) VALUE 1.
                        01 J PIC 9(3) VALUE 1.
                        PROCEDURE DIVISION.
                        P1.
                            MOVE Function REVERSE(InputString) TO OutputString
                            DISPLAY OutputString
                            STOP RUN.
                        FUNCTION REVERSE(A) RETURNS B
                            LOCAL A, B.
                            PERFORM VARYING I FROM LENGTH OF A BY -1 UNTIL I = 0
                                MOVE FUNCTION SUBSTRING(A, I, 1) TO B(I)
                            END-PERFORM
                            SET B TO FUNCTION TRIM(B)
                        END FUNCTION.""",
        "Java Translation": """public class ReverseString {
                                    public static void main(String[] args) {
                                        String inputString = "Hello, World!";
                                        StringBuilder outputString = new StringBuilder();
                                        for (int i = inputString.length() - 1; i >= 0; i--) {
                                            outputString.append(inputString.charAt(i));
                                        }
                                        System.out.println(outputString);
                                    }
                                }"""
    },
    {
        "ID": 8,
        "Domain": "Sorting",
        "Purpose": "BubbleSort",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. BubbleSort.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Array OCCURS 10 TIMES PIC 9(3) VALUE 99, 88, 77, 66, 55, 44, 33, 22, 11, 00.
                        01 N PIC 9(3) VALUE 10.
                        01 I PIC 9(3).
                        01 J PIC 9(3).
                        01 TEMP PIC 9(3).
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > N - 1
                                PERFORM VARYING J FROM 1 BY 1 UNTIL J > N - I
                                    IF Array(J) > Array(J + 1)
                                        MOVE Array(J) TO TEMP
                                        MOVE Array(J + 1) TO Array(J)
                                        MOVE TEMP TO Array(J + 1)
                                    END-IF
                                END-PERFORM
                            END-PERFORM
                            DISPLAY "Sorted Array:"
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I <= N
                                DISPLAY Array(I)
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class BubbleSort {
                                    public static void main(String[] args) {
                                        int[] array = {99, 88, 77, 66, 55, 44, 33, 22, 11, 00};
                                        int n = array.length;
                                        int temp;
                                        for (int i = 0; i < n - 1; i++) {
                                            for (int j = 0; j < n - i - 1; j++) {
                                                if (array[j] > array[j + 1]) {
                                                    temp = array[j];
                                                    array[j] = array[j + 1];
                                                    array[j + 1] = temp;
                                                }
                                            }
                                        }
                                        System.out.println("Sorted Array:");
                                        for (int num : array) {
                                            System.out.println(num);
                                        }
                                    }
                                }"""
    },
    {
        "ID": 9,
        "Domain": "Data Structures",
        "Purpose": "Linked List",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. LinkedList.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Node.
                            02 Data PIC X(10).
                            02 NextPointer POINTER.
                        01 HeadPointer POINTER.
                        PROCEDURE DIVISION.
                        P1.
                            CALL "INIT_LIST"
                            CALL "INSERT" USING "Data1"
                            CALL "INSERT" USING "Data2"
                            CALL "INSERT" USING "Data3"
                            CALL "DISPLAY_LIST"
                            CALL "DELETE" USING "Data2"
                            CALL "DISPLAY_LIST"
                            STOP RUN.
                        INIT_LIST.
                            SET HeadPointer TO NULL
                            EXIT PROGRAM.
                        INSERT USING Item.
                            SET Node TO ADDRESS OF Node
                            MOVE Item TO Data OF Node
                            MOVE HeadPointer TO NextPointer OF Node
                            SET HeadPointer TO Node
                            EXIT PROGRAM.
                        DISPLAY_LIST.
                            SET Node TO HeadPointer
                            PERFORM UNTIL Node = NULL
                                DISPLAY Data OF Node
                                SET Node TO NextPointer OF Node
                            END-PERFORM
                            EXIT PROGRAM.
                        DELETE USING Item.
                            SET Node TO HeadPointer
                            SET PrevNode TO NULL
                            PERFORM UNTIL Node = NULL
                                IF Data OF Node = Item
                                    IF PrevNode = NULL
                                        SET HeadPointer TO NextPointer OF Node
                                    ELSE
                                        SET NextPointer OF PrevNode TO NextPointer OF Node
                                    END-IF
                                    EXIT PROGRAM
                                END-IF
                                SET PrevNode TO Node
                                SET Node TO NextPointer OF Node
                            END-PERFORM
                            EXIT PROGRAM.""",
        "Java Translation": """public class LinkedList {
                                    static class Node {
                                        String data;
                                        Node next;

                                        Node(String data) {
                                            this.data = data;
                                            this.next = null;
                                        }
                                    }

                                    Node head;

                                    LinkedList() {
                                        this.head = null;
                                    }

                                    void insert(String item) {
                                        Node newNode = new Node(item);
                                        newNode.next = head;
                                        head = newNode;
                                    }

                                    void display() {
                                        Node current = head;
                                        while (current != null) {
                                            System.out.println(current.data);
                                            current = current.next;
                                        }
                                    }

                                    void delete(String item) {
                                        Node current = head;
                                        Node prev = null;
                                        while (current != null) {
                                            if (current.data.equals(item)) {
                                                if (prev == null) {
                                                    head = current.next;
                                                } else {
                                                    prev.next = current.next;
                                                }
                                                return;
                                            }
                                            prev = current;
                                            current = current.next;
                                        }
                                    }

                                    public static void main(String[] args) {
                                        LinkedList list = new LinkedList();
                                        list.insert("Data1");
                                        list.insert("Data2");
                                        list.insert("Data3");
                                        list.display();
                                        list.delete("Data2");
                                        list.display();
                                    }
                                }"""
    },
    {
        "ID": 10,
        "Domain": "Networking",
        "Purpose": "TCP Client",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. TCPClient.
                        ENVIRONMENT DIVISION.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Server-IP PIC X(15) VALUE "127.0.0.1".
                        01 PortNumber PIC 9(4) VALUE 8080.
                        01 Client-Socket USAGE POINTER.
                        01 Buffer PIC X(255).
                        PROCEDURE DIVISION.
                        P1.
                            CALL "CONNECT_TO_SERVER" USING Server-IP PortNumber
                            CALL "SEND_DATA" USING "Hello, Server!"
                            CALL "RECEIVE_DATA"
                            CALL "CLOSE_CONNECTION"
                            STOP RUN.
                        CONNECT_TO_SERVER USING ServerIP PortNo.
                            CALL "WSOCK32.DLL" USING BY VALUE 2, BY REFERENCE Client-Socket
                            CALL "WSOCK32.DLL" USING BY VALUE 20, BY VALUE ServerIP, BY VALUE PortNo, BY VALUE Client-Socket
                            EXIT PROGRAM.
                        SEND_DATA USING Data.
                            STRING Data DELIMITED BY SIZE INTO Buffer
                            CALL "WSOCK32.DLL" USING BY VALUE 1, BY VALUE Buffer, BY VALUE LENGTH OF Buffer, BY VALUE Client-Socket
                            EXIT PROGRAM.
                        RECEIVE_DATA.
                            CALL "WSOCK32.DLL" USING BY VALUE 0, BY VALUE Buffer, BY VALUE LENGTH OF Buffer, BY VALUE Client-Socket
                            DISPLAY Buffer
                            EXIT PROGRAM.
                        CLOSE_CONNECTION.
                            CALL "WSOCK32.DLL" USING BY VALUE 6, BY VALUE Client-Socket
                            EXIT PROGRAM.""",
        "Java Translation": """import java.io.*;
                                import java.net.*;

                                public class TCPClient {
                                    public static void main(String[] args) {
                                        try {
                                            String serverIP = "127.0.0.1";
                                            int portNumber = 8080;
                                            Socket clientSocket = new Socket(serverIP, portNumber);
                                            PrintWriter out = new PrintWriter(clientSocket.getOutputStream(), true);
                                            BufferedReader in = new BufferedReader(new InputStreamReader(clientSocket.getInputStream()));

                                            out.println("Hello, Server!");
                                            String response = in.readLine();
                                            System.out.println(response);

                                            out.close();
                                            in.close();
                                            clientSocket.close();
                                        } catch (IOException e) {
                                            e.printStackTrace();
                                        }
                                    }
                                }"""
    },
    {
        "ID": 11,
        "Domain": "Database Management",
        "Purpose": "CRUD Operations",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CRUDOperations.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 EmployeeRecord.
                            02 EmployeeID PIC X(5).
                            02 EmployeeName PIC X(30).
                            02 EmployeeAge PIC 99.
                        01 DatabaseFile.
                            02 DatabaseRecord OCCURS 100 TIMES DEPENDING ON EmployeeCount.
                                03 EmployeeID PIC X(5).
                                03 EmployeeName PIC X(30).
                                03 EmployeeAge PIC 99.
                        01 EmployeeCount PIC 99 VALUE 0.
                        PROCEDURE DIVISION.
                        P1.
                            DISPLAY "1. Add Employee"
                            DISPLAY "2. View Employees"
                            DISPLAY "3. Update Employee"
                            DISPLAY "4. Delete Employee"
                            DISPLAY "5. Exit"
                            ACCEPT Choice
                            EVALUATE Choice
                                WHEN 1
                                    CALL "ADD_EMPLOYEE"
                                WHEN 2
                                    CALL "VIEW_EMPLOYEES"
                                WHEN 3
                                    CALL "UPDATE_EMPLOYEE"
                                WHEN 4
                                    CALL "DELETE_EMPLOYEE"
                                WHEN 5
                                    STOP RUN
                                WHEN OTHER
                                    DISPLAY "Invalid Choice"
                            END-EVALUATE
                            GO TO P1.
                        ADD_EMPLOYEE.
                            ACCEPT EmployeeID FROM STDIN
                            ACCEPT EmployeeName FROM STDIN
                            ACCEPT EmployeeAge FROM STDIN
                            ADD 1 TO EmployeeCount
                            MOVE EmployeeID TO EmployeeRecord(EmployeeCount)
                            MOVE EmployeeName TO EmployeeRecord(EmployeeCount)
                            MOVE EmployeeAge TO EmployeeRecord(EmployeeCount)
                            EXIT PROGRAM.
                        VIEW_EMPLOYEES.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > EmployeeCount
                                DISPLAY DatabaseRecord(I)
                            END-PERFORM
                            EXIT PROGRAM.
                        UPDATE_EMPLOYEE.
                            ACCEPT EmployeeID FROM STDIN
                            ACCEPT EmployeeName FROM STDIN
                            ACCEPT EmployeeAge FROM STDIN
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > EmployeeCount
                                IF EmployeeRecord(I) = EmployeeID
                                    MOVE EmployeeName TO DatabaseRecord(I)
                                    MOVE EmployeeAge TO DatabaseRecord(I)
                                    EXIT PROGRAM
                                END-IF
                            END-PERFORM
                            EXIT PROGRAM.
                        DELETE_EMPLOYEE.
                            ACCEPT EmployeeID FROM STDIN
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > EmployeeCount
                                IF EmployeeRecord(I) = EmployeeID
                                    UNSTRING DatabaseRecord(I) INTO EmployeeID, EmployeeName, EmployeeAge
                                    SUBTRACT 1 FROM EmployeeCount
                                    EXIT PROGRAM
                                END-IF
                            END-PERFORM
                            EXIT PROGRAM.""",
        "Java Translation": """import java.util.ArrayList;
                                import java.util.Scanner;

                                public class CRUDOperations {
                                    static class Employee {
                                        String employeeID;
                                        String employeeName;
                                        int employeeAge;

                                        Employee(String employeeID, String employeeName, int employeeAge) {
                                            this.employeeID = employeeID;
                                            this.employeeName = employeeName;
                                            this.employeeAge = employeeAge;
                                        }
                                    }

                                    static ArrayList<Employee> database = new ArrayList<>();

                                    public static void main(String[] args) {
                                        Scanner scanner = new Scanner(System.in);
                                        while (true) {
                                            System.out.println("1. Add Employee");
                                            System.out.println("2. View Employees");
                                            System.out.println("3. Update Employee");
                                            System.out.println("4. Delete Employee");
                                            System.out.println("5. Exit");
                                            int choice = scanner.nextInt();
                                            switch (choice) {
                                                case 1:
                                                    addEmployee();
                                                    break;
                                                case 2:
                                                    viewEmployees();
                                                    break;
                                                case 3:
                                                    updateEmployee();
                                                    break;
                                                case 4:
                                                    deleteEmployee();
                                                    break;
                                                case 5:
                                                    System.exit(0);
                                                default:
                                                    System.out.println("Invalid Choice");
                                            }
                                        }
                                    }

                                    static void addEmployee() {
                                        Scanner scanner = new Scanner(System.in);
                                        System.out.print("Enter Employee ID: ");
                                        String employeeID = scanner.nextLine();
                                        System.out.print("Enter Employee Name: ");
                                        String employeeName = scanner.nextLine();
                                        System.out.print("Enter Employee Age: ");
                                        int employeeAge = scanner.nextInt();
                                        database.add(new Employee(employeeID, employeeName, employeeAge));
                                    }

                                    static void viewEmployees() {
                                        for (Employee employee : database) {
                                            System.out.println(employee.employeeID + " | " + employee.employeeName + " | " + employee.employeeAge);
                                        }
                                    }

                                    static void updateEmployee() {
                                        Scanner scanner = new Scanner(System.in);
                                        System.out.print("Enter Employee ID to Update: ");
                                        String employeeID = scanner.nextLine();
                                        for (Employee employee : database) {
                                            if (employee.employeeID.equals(employeeID)) {
                                                System.out.print("Enter New Employee Name: ");
                                                String employeeName = scanner.nextLine();
                                                System.out.print("Enter New Employee Age: ");
                                                int employeeAge = scanner.nextInt();
                                                employee.employeeName = employeeName;
                                                employee.employeeAge = employeeAge;
                                                return;
                                            }
                                        }
                                        System.out.println("Employee ID not found.");
                                    }

                                    static void deleteEmployee() {
                                        Scanner scanner = new Scanner(System.in);
                                        System.out.print("Enter Employee ID to Delete: ");
                                        String employeeID = scanner.nextLine();
                                        for (Employee employee : database) {
                                            if (employee.employeeID.equals(employeeID)) {
                                                database.remove(employee);
                                                return;
                                            }
                                        }
                                        System.out.println("Employee ID not found.");
                                    }
                                }"""
    },
        {
        "ID": 12,
        "Domain": "Multithreading",
        "Purpose": "Banking System",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. BankingSystem.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 AccountBalance PIC 99999 VALUE 10000.
                        01 WithdrawalAmount PIC 99999.
                        01 DepositAmount PIC 99999.
                        PROCEDURE DIVISION.
                        P1.
                            DISPLAY "Current Balance: " AccountBalance
                            ACCEPT WithdrawalAmount FROM STDIN
                            IF WithdrawalAmount <= AccountBalance
                                SUBTRACT WithdrawalAmount FROM AccountBalance GIVING AccountBalance
                                DISPLAY "Withdrawal Successful"
                            ELSE
                                DISPLAY "Insufficient Funds"
                            END-IF
                            ACCEPT DepositAmount FROM STDIN
                            ADD DepositAmount TO AccountBalance GIVING AccountBalance
                            DISPLAY "Deposit Successful"
                            STOP RUN.""",
        "Java Translation": """import java.util.Scanner;

                                public class BankingSystem {
                                    static int accountBalance = 10000;

                                    public static void main(String[] args) {
                                        Scanner scanner = new Scanner(System.in);
                                        while (true) {
                                            System.out.println("Current Balance: " + accountBalance);
                                            System.out.print("Enter Withdrawal Amount: ");
                                            int withdrawalAmount = scanner.nextInt();
                                            if (withdrawalAmount <= accountBalance) {
                                                accountBalance -= withdrawalAmount;
                                                System.out.println("Withdrawal Successful");
                                            } else {
                                                System.out.println("Insufficient Funds");
                                            }
                                            System.out.print("Enter Deposit Amount: ");
                                            int depositAmount = scanner.nextInt();
                                            accountBalance += depositAmount;
                                            System.out.println("Deposit Successful");
                                        }
                                    }
                                }"""
    },
    {
        "ID": 13,
        "Domain": "Machine Learning",
        "Purpose": "Linear Regression",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. LinearRegression.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 X-Values OCCURS 10 TIMES PIC 999 VALUE 1, 2, 3, 4, 5, 6, 7, 8, 9, 10.
                        01 Y-Values OCCURS 10 TIMES PIC 999 VALUE 5, 7, 9, 11, 13, 15, 17, 19, 21, 23.
                        01 X-Mean PIC 999.
                        01 Y-Mean PIC 999.
                        01 Sum-X-X-Mean PIC 999.
                        01 Sum-Y-Y-Mean PIC 999.
                        01 Sum-X-Y PIC 999.
                        01 Sum-X-Squared PIC 999.
                        01 B1 PIC 9(3)V99.
                        01 B0 PIC 9(3)V99.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > 10
                                ADD X-Values(I) TO Sum-X-X-Mean
                                ADD Y-Values(I) TO Sum-Y-Y-Mean
                            END-PERFORM
                            COMPUTE X-Mean = Sum-X-X-Mean / 10
                            COMPUTE Y-Mean = Sum-Y-Y-Mean / 10
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > 10
                                COMPUTE Sum-X-Y = Sum-X-Y + ((X-Values(I) - X-Mean) * (Y-Values(I) - Y-Mean))
                                COMPUTE Sum-X-Squared = Sum-X-Squared + ((X-Values(I) - X-Mean) ** 2)
                            END-PERFORM
                            COMPUTE B1 = Sum-X-Y / Sum-X-Squared
                            COMPUTE B0 = Y-Mean - (B1 * X-Mean)
                            DISPLAY "Linear Regression Equation: Y = " B1 "X + " B0
                            STOP RUN.""",
        "Java Translation": """public class LinearRegression {
                                    public static void main(String[] args) {
                                        int[] xValues = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10};
                                        int[] yValues = {5, 7, 9, 11, 13, 15, 17, 19, 21, 23};
                                        int sumX = 0, sumY = 0, sumXY = 0, sumXSquared = 0;

                                        for (int i = 0; i < 10; i++) {
                                            sumX += xValues[i];
                                            sumY += yValues[i];
                                        }
                                        double xMean = sumX / 10.0;
                                        double yMean = sumY / 10.0;

                                        for (int i = 0; i < 10; i++) {
                                            sumXY += (xValues[i] - xMean) * (yValues[i] - yMean);
                                            sumXSquared += Math.pow(xValues[i] - xMean, 2);
                                        }

                                        double b1 = sumXY / sumXSquared;
                                        double b0 = yMean - (b1 * xMean);

                                        System.out.println("Linear Regression Equation: Y = " + b1 + "X + " + b0);
                                    }
                                }"""
    },
    {
        "ID": 14,
        "Domain": "Web Development",
        "Purpose": "Simple Web Server",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. SimpleWebServer.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 ServerSocket USAGE POINTER.
                        01 ClientSocket USAGE POINTER.
                        01 InStream USAGE POINTER.
                        01 OutStream USAGE POINTER.
                        01 Request PIC X(255).
                        01 Response PIC X(255).
                        PROCEDURE DIVISION.
                        P1.
                            CALL "CREATE_SERVER" USING BY REFERENCE ServerSocket
                            PERFORM UNTIL 1 = 0
                                CALL "ACCEPT_CONNECTION" USING BY REFERENCE ServerSocket BY REFERENCE ClientSocket
                                CALL "RECEIVE_REQUEST" USING BY REFERENCE ClientSocket BY REFERENCE InStream
                                CALL "PROCESS_REQUEST" USING BY REFERENCE Request BY REFERENCE Response
                                CALL "SEND_RESPONSE" USING BY REFERENCE ClientSocket BY REFERENCE OutStream
                                CALL "CLOSE_CONNECTION" USING BY REFERENCE ClientSocket
                            END-PERFORM
                            CALL "CLOSE_SERVER" USING BY REFERENCE ServerSocket
                            STOP RUN.
                        CREATE_SERVER USING Socket.
                            CALL "WSOCK32.DLL" USING BY VALUE 2, BY REFERENCE Socket
                            CALL "WSOCK32.DLL" USING BY VALUE 20, BY VALUE "127.0.0.1", BY VALUE 8080, BY REFERENCE Socket
                            EXIT PROGRAM.
                        ACCEPT_CONNECTION USING ServerSocket, ClientSocket.
                            CALL "WSOCK32.DLL" USING BY VALUE 3, BY REFERENCE ServerSocket, BY REFERENCE ClientSocket
                            EXIT PROGRAM.
                        RECEIVE_REQUEST USING ClientSocket, InStream.
                            CALL "WSOCK32.DLL" USING BY VALUE 0, BY REFERENCE Request, BY VALUE LENGTH OF Request, BY REFERENCE ClientSocket
                            EXIT PROGRAM.
                        PROCESS_REQUEST USING Request, Response.
                            MOVE "HTTP/1.1 200 OK" TO Response
                            STRING "Content-Type: text/html" DELIMITED BY SIZE INTO Response
                            STRING "" DELIMITED BY SIZE INTO Response
                            STRING "<html><body><h1>Hello, World!</h1></body></html>" DELIMITED BY SIZE INTO Response
                            EXIT PROGRAM.
                        SEND_RESPONSE USING ClientSocket, OutStream.
                            CALL "WSOCK32.DLL" USING BY VALUE 1, BY REFERENCE Response, BY VALUE LENGTH OF Response, BY REFERENCE ClientSocket
                            EXIT PROGRAM.
                        CLOSE_CONNECTION USING Socket.
                            CALL "WSOCK32.DLL" USING BY VALUE 6, BY REFERENCE Socket
                            EXIT PROGRAM.
                        CLOSE_SERVER USING Socket.
                            CALL "WSOCK32.DLL" USING BY VALUE 6, BY REFERENCE Socket
                            EXIT PROGRAM.""",
        "Java Translation": """import java.io.*;
                                import java.net.*;

                                public class SimpleWebServer {
                                    public static void main(String[] args) {
                                        try {
                                            ServerSocket serverSocket = new ServerSocket(8080);
                                            System.out.println("Server started");
                                            while (true) {
                                                Socket clientSocket = serverSocket.accept();
                                                System.out.println("Client connected: " + clientSocket.getInetAddress());

                                                BufferedReader inStream = new BufferedReader(new InputStreamReader(clientSocket.getInputStream()));
                                                String request = inStream.readLine();
                                                System.out.println("Request: " + request);

                                                OutputStream outStream = clientSocket.getOutputStream();
                                                PrintWriter out = new PrintWriter(outStream, true);

                                                out.println("HTTP/1.1 200 OK");
                                                out.println("Content-Type: text/html");
                                                out.println("");
                                                out.println("<html><body><h1>Hello, World!</h1></body></html>");

                                                out.close();
                                                inStream.close();
                                                clientSocket.close();
                                            }
                                        } catch (IOException e) {
                                            e.printStackTrace();
                                        }
                                    }
                                }"""
    },
    {
        "ID": 15,
        "Domain": "Artificial Intelligence",
        "Purpose": "Chess Game",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. ChessGame.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Board OCCURS 64 TIMES.
                            02 Piece PIC X(2).
                        01 Player1Piece PIC X(2) VALUE "WP".
                        01 Player2Piece PIC X(2) VALUE "BP".
                        PROCEDURE DIVISION.
                        P1.
                            CALL "INITIALIZE_BOARD"
                            CALL "DISPLAY_BOARD"
                            CALL "MOVE" USING 7 6
                            CALL "DISPLAY_BOARD"
                            CALL "MOVE" USING 0 2
                            CALL "DISPLAY_BOARD"
                            STOP RUN.
                        INITIALIZE_BOARD.
                            MOVE " " TO Board
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > 8
                                MOVE Player1Piece TO Board(I)
                                MOVE Player1Piece TO Board(8 * 1 + I)
                                MOVE Player2Piece TO Board(8 * 7 + I)
                            END-PERFORM
                            EXIT PROGRAM.
                        DISPLAY_BOARD.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > 64
                                DISPLAY Board(I)
                                IF I MOD 8 = 0
                                    DISPLAY ""
                                END-IF
                            END-PERFORM
                            EXIT PROGRAM.
                        MOVE USING FromSquare ToSquare.
                            MOVE Board(FromSquare) TO Board(ToSquare)
                            MOVE " " TO Board(FromSquare)
                            EXIT PROGRAM.""",
        "Java Translation": """public class ChessGame {
                                    static String[] board = new String[64];
                                    static String player1Piece = "WP";
                                    static String player2Piece = "BP";

                                    public static void main(String[] args) {
                                        initializeBoard();
                                        displayBoard();
                                        move(7, 6);
                                        displayBoard();
                                        move(0, 2);
                                        displayBoard();
                                    }

                                    static void initializeBoard() {
                                        for (int i = 0; i < 64; i++) {
                                            board[i] = " ";
                                        }
                                        for (int i = 0; i < 8; i++) {
                                            board[i] = player1Piece;
                                            board[8 * 1 + i] = player1Piece;
                                            board[8 * 7 + i] = player2Piece;
                                        }
                                    }

                                    static void displayBoard() {
                                        for (int i = 0; i < 64; i++) {
                                            System.out.print(board[i] + " ");
                                            if ((i + 1) % 8 == 0) {
                                                System.out.println();
                                            }
                                        }
                                    }

                                    static void move(int fromSquare, int toSquare) {
                                        board[toSquare] = board[fromSquare];
                                        board[fromSquare] = " ";
                                    }
                                }"""
    },
    {
        "ID": 16,
        "Domain": "Data Structures",
        "Purpose": "Binary Search Tree",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. BinarySearchTree.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 TreeNode.
                            02 Value PIC 999.
                            02 LeftChild USAGE POINTER.
                            02 RightChild USAGE POINTER.
                        01 RootNode USAGE POINTER.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > 10
                                CALL "INSERT_NODE" USING I
                            END-PERFORM
                            CALL "DISPLAY_IN_ORDER" USING RootNode
                            STOP RUN.
                        INSERT_NODE USING NewValue.
                            CALL "ALLOCATE_NODE" USING NewNode
                            MOVE NewValue TO NewNode.Value
                            IF RootNode = NULL
                                MOVE NewNode TO RootNode
                            ELSE
                                PERFORM UNTIL CurrentNode = NULL
                                    MOVE CurrentNode TO ParentNode
                                    IF NewValue < CurrentNode.Value
                                        MOVE CurrentNode.LeftChild TO CurrentNode
                                    ELSE
                                        MOVE CurrentNode.RightChild TO CurrentNode
                                    END-IF
                                END-PERFORM
                                IF NewValue < ParentNode.Value
                                    MOVE NewNode TO ParentNode.LeftChild
                                ELSE
                                    MOVE NewNode TO ParentNode.RightChild
                                END-IF
                            END-IF
                            EXIT PROGRAM.
                        DISPLAY_IN_ORDER USING Node.
                            IF Node NOT = NULL
                                CALL "DISPLAY_IN_ORDER" USING Node.LeftChild
                                DISPLAY Node.Value
                                CALL "DISPLAY_IN_ORDER" USING Node.RightChild
                            END-IF
                            EXIT PROGRAM.
                        ALLOCATE_NODE USING NewNode.
                            CALL "WSOCK32.DLL" USING BY VALUE 21, BY VALUE SIZEOF TreeNode, BY REFERENCE NewNode
                            MOVE NULL TO NewNode.LeftChild
                            MOVE NULL TO NewNode.RightChild
                            EXIT PROGRAM.""",
        "Java Translation": """class TreeNode {
                                    int value;
                                    TreeNode leftChild, rightChild;

                                    TreeNode(int value) {
                                        this.value = value;
                                        leftChild = rightChild = null;
                                    }
                                }

                                public class BinarySearchTree {
                                    TreeNode root;

                                    BinarySearchTree() {
                                        root = null;
                                    }

                                    void insertNode(int newValue) {
                                        root = insertNodeRecursive(root, newValue);
                                    }

                                    TreeNode insertNodeRecursive(TreeNode root, int newValue) {
                                        if (root == null) {
                                            root = new TreeNode(newValue);
                                            return root;
                                        }
                                        if (newValue < root.value) {
                                            root.leftChild = insertNodeRecursive(root.leftChild, newValue);
                                        } else if (newValue > root.value) {
                                            root.rightChild = insertNodeRecursive(root.rightChild, newValue);
                                        }
                                        return root;
                                    }

                                    void displayInOrder(TreeNode node) {
                                        if (node != null) {
                                            displayInOrder(node.leftChild);
                                            System.out.print(node.value + " ");
                                            displayInOrder(node.rightChild);
                                        }
                                    }

                                    public static void main(String[] args) {
                                        BinarySearchTree bst = new BinarySearchTree();
                                        for (int i = 1; i <= 10; i++) {
                                            bst.insertNode(i);
                                        }
                                        bst.displayInOrder(bst.root);
                                    }
                                }"""
    },
    {
        "ID": 17,
        "Domain": "Network Programming",
        "Purpose": "TCP Client-Server Communication",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. TCPClientServer.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 ClientSocket USAGE POINTER.
                        01 ServerSocket USAGE POINTER.
                        01 InStream USAGE POINTER.
                        01 OutStream USAGE POINTER.
                        01 MessageToSend PIC X(255) VALUE "Hello, Server!".
                        01 ReceivedMessage PIC X(255).
                        PROCEDURE DIVISION.
                        P1.
                            CALL "CREATE_SERVER" USING BY REFERENCE ServerSocket
                            CALL "CREATE_CLIENT" USING BY REFERENCE ClientSocket
                            CALL "SEND_MESSAGE" USING BY REFERENCE ClientSocket BY REFERENCE OutStream
                            CALL "RECEIVE_MESSAGE" USING BY REFERENCE ServerSocket BY REFERENCE InStream
                            CALL "CLOSE_CLIENT" USING BY REFERENCE ClientSocket
                            CALL "CLOSE_SERVER" USING BY REFERENCE ServerSocket
                            STOP RUN.
                        CREATE_SERVER USING ServerSocket.
                            CALL "WSOCK32.DLL" USING BY VALUE 2, BY REFERENCE ServerSocket
                            CALL "WSOCK32.DLL" USING BY VALUE 20, BY VALUE "127.0.0.1", BY VALUE 8080, BY REFERENCE ServerSocket
                            CALL "WSOCK32.DLL" USING BY VALUE 3, BY REFERENCE ServerSocket, BY REFERENCE ServerSocket
                            EXIT PROGRAM.
                        CREATE_CLIENT USING ClientSocket.
                            CALL "WSOCK32.DLL" USING BY VALUE 2, BY REFERENCE ClientSocket
                            CALL "WSOCK32.DLL" USING BY VALUE 20, BY VALUE "127.0.0.1", BY VALUE 8080, BY REFERENCE ClientSocket
                            CALL "WSOCK32.DLL" USING BY VALUE 3, BY REFERENCE ClientSocket, BY REFERENCE ClientSocket
                            EXIT PROGRAM.
                        SEND_MESSAGE USING Socket, Stream.
                            CALL "WSOCK32.DLL" USING BY VALUE 1, BY REFERENCE MessageToSend, BY VALUE LENGTH OF MessageToSend, BY REFERENCE Stream
                            EXIT PROGRAM.
                        RECEIVE_MESSAGE USING Socket, Stream.
                            CALL "WSOCK32.DLL" USING BY VALUE 0, BY REFERENCE ReceivedMessage, BY VALUE LENGTH OF ReceivedMessage, BY REFERENCE Stream
                            DISPLAY ReceivedMessage
                            EXIT PROGRAM.
                        CLOSE_CLIENT USING Socket.
                            CALL "WSOCK32.DLL" USING BY VALUE 6, BY REFERENCE Socket
                            EXIT PROGRAM.
                        CLOSE_SERVER USING Socket.
                            CALL "WSOCK32.DLL" USING BY VALUE 6, BY REFERENCE Socket
                            EXIT PROGRAM.""",
        "Java Translation": """import java.io.*;
                                import java.net.*;

                                public class TCPClientServer {
                                    static final String SERVER_IP = "127.0.0.1";
                                    static final int SERVER_PORT = 8080;

                                    public static void main(String[] args) {
                                        try {
                                            Thread serverThread = new Thread(() -> {
                                                try {
                                                    ServerSocket serverSocket = new ServerSocket(SERVER_PORT);
                                                    Socket clientSocket = serverSocket.accept();
                                                    System.out.println("Client connected: " + clientSocket.getInetAddress());
                                                    BufferedReader inStream = new BufferedReader(new InputStreamReader(clientSocket.getInputStream()));
                                                    String receivedMessage = inStream.readLine();
                                                    System.out.println("Received from client: " + receivedMessage);
                                                    PrintWriter outStream = new PrintWriter(clientSocket.getOutputStream(), true);
                                                    outStream.println("Hello, Client!");
                                                    inStream.close();
                                                    outStream.close();
                                                    clientSocket.close();
                                                    serverSocket.close();
                                                } catch (IOException e) {
                                                    e.printStackTrace();
                                                }
                                            });
                                            serverThread.start();

                                            Thread.sleep(1000); // Give the server some time to start

                                            Socket clientSocket = new Socket(SERVER_IP, SERVER_PORT);
                                            System.out.println("Connected to server: " + clientSocket.getInetAddress());
                                            BufferedReader inStream = new BufferedReader(new InputStreamReader(clientSocket.getInputStream()));
                                            PrintWriter outStream = new PrintWriter(clientSocket.getOutputStream(), true);
                                            outStream.println("Hello, Server!");
                                            String receivedMessage = inStream.readLine();
                                            System.out.println("Received from server: " + receivedMessage);
                                            inStream.close();
                                            outStream.close();
                                            clientSocket.close();
                                        } catch (IOException | InterruptedException e) {
                                            e.printStackTrace();
                                        }
                                    }
                                }"""
    },
        {
        "ID": 18,
        "Domain": "Encryption",
        "Purpose": "RSA Algorithm",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. RSAAlgorithm.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 P PIC 999 VALUE 61.
                        01 Q PIC 999 VALUE 53.
                        01 N PIC 9999 VALUE 3233.
                        01 PhiN PIC 9999 VALUE 3120.
                        01 E PIC 999 VALUE 17.
                        01 D PIC 999 VALUE 2753.
                        01 Message PIC 9999 VALUE 1234.
                        01 EncryptedMessage PIC 9999.
                        01 DecryptedMessage PIC 9999.
                        PROCEDURE DIVISION.
                        P1.
                            COMPUTE EncryptedMessage = MODEXP(Message, E, N)
                            COMPUTE DecryptedMessage = MODEXP(EncryptedMessage, D, N)
                            DISPLAY "Original Message: " Message
                            DISPLAY "Encrypted Message: " EncryptedMessage
                            DISPLAY "Decrypted Message: " DecryptedMessage
                            STOP RUN.
                        MODEXP USING Base Exponent Modulus.
                            COMPUTE Result = 1
                            PERFORM UNTIL Exponent = 0
                                IF MOD(Exponent, 2) = 1
                                    COMPUTE Result = MOD(Result * Base, Modulus)
                                END-IF
                                COMPUTE Base = MOD(Base * Base, Modulus)
                                COMPUTE Exponent = Exponent / 2
                            END-PERFORM
                            EXIT PROGRAM.""",
        "Java Translation": """public class RSAAlgorithm {
                                    public static void main(String[] args) {
                                        int p = 61;
                                        int q = 53;
                                        int n = p * q;
                                        int phiN = (p - 1) * (q - 1);
                                        int e = 17;
                                        int d = 2753;
                                        int message = 1234;
                                        int encryptedMessage = modExp(message, e, n);
                                        int decryptedMessage = modExp(encryptedMessage, d, n);
                                        System.out.println("Original Message: " + message);
                                        System.out.println("Encrypted Message: " + encryptedMessage);
                                        System.out.println("Decrypted Message: " + decryptedMessage);
                                    }

                                    static int modExp(int base, int exponent, int modulus) {
                                        int result = 1;
                                        while (exponent > 0) {
                                            if (exponent % 2 == 1) {
                                                result = (result * base) % modulus;
                                            }
                                            base = (base * base) % modulus;
                                            exponent /= 2;
                                        }
                                        return result;
                                    }
                                }"""
    },
    {
        "ID": 19,
        "Domain": "Graphics",
        "Purpose": "Simple Graphics Rendering",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. GraphicsRendering.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 ScreenWidth PIC 9999 VALUE 80.
                        01 ScreenHeight PIC 9999 VALUE 24.
                        01 ScreenBuffer OCCURS 1920 TIMES.
                            02 PixelValue PIC X(2) VALUE "  ".
                        PROCEDURE DIVISION.
                        P1.
                            CALL "CLEAR_SCREEN"
                            CALL "DRAW_LINE" USING 0 0 79 23
                            CALL "DRAW_RECTANGLE" USING 10 5 30 15
                            CALL "DISPLAY_SCREEN"
                            STOP RUN.
                        CLEAR_SCREEN.
                            MOVE "  " TO ScreenBuffer
                            PERFORM VARYING I FROM 2 BY 2 UNTIL I > 1920
                                MOVE "  " TO ScreenBuffer(I)
                            END-PERFORM
                            EXIT PROGRAM.
                        DRAW_LINE USING X1 Y1 X2 Y2.
                            COMPUTE DeltaX = X2 - X1
                            COMPUTE DeltaY = Y2 - Y1
                            COMPUTE Slope = DeltaY / DeltaX
                            IF ABS(Slope) <= 1
                                PERFORM VARYING X FROM X1 BY 1 UNTIL X > X2
                                    COMPUTE Y = Y1 + (Slope * (X - X1))
                                    MOVE "##" TO ScreenBuffer((Y - 1) * ScreenWidth + X)
                                END-PERFORM
                            ELSE
                                COMPUTE Slope = 1 / Slope
                                PERFORM VARYING Y FROM Y1 BY 1 UNTIL Y > Y2
                                    COMPUTE X = X1 + (Slope * (Y - Y1))
                                    MOVE "##" TO ScreenBuffer((Y - 1) * ScreenWidth + X)
                                END-PERFORM
                            END-IF
                            EXIT PROGRAM.
                        DRAW_RECTANGLE USING X1 Y1 X2 Y2.
                            CALL "DRAW_LINE" USING X1 Y1 X1 Y2
                            CALL "DRAW_LINE" USING X1 Y1 X2 Y1
                            CALL "DRAW_LINE" USING X2 Y1 X2 Y2
                            CALL "DRAW_LINE" USING X1 Y2 X2 Y2
                            EXIT PROGRAM.
                        DISPLAY_SCREEN.
                            DISPLAY "┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐"
                            PERFORM VARYING Y FROM 1 BY 1 UNTIL Y > ScreenHeight
                                DISPLAY "│"
                                PERFORM VARYING X FROM 1 BY 1 UNTIL X > ScreenWidth
                                    DISPLAY ScreenBuffer((Y - 1) * ScreenWidth + X)
                                END-PERFORM
                                DISPLAY "│"
                            END-PERFORM
                            DISPLAY "└──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘"
                            EXIT PROGRAM.""",
        "Java Translation": """public class GraphicsRendering {
                                    static final int SCREEN_WIDTH = 80;
                                    static final int SCREEN_HEIGHT = 24;
                                    static final String[] screenBuffer = new String[SCREEN_WIDTH * SCREEN_HEIGHT];

                                    public static void main(String[] args) {
                                        clearScreen();
                                        drawLine(0, 0, 79, 23);
                                        drawRectangle(10, 5, 30, 15);
                                        displayScreen();
                                    }

                                    static void clearScreen() {
                                        for (int i = 0; i < SCREEN_WIDTH * SCREEN_HEIGHT; i++) {
                                            screenBuffer[i] = "  ";
                                        }
                                    }

                                    static void drawLine(int x1, int y1, int x2, int y2) {
                                        int deltaX = x2 - x1;
                                        int deltaY = y2 - y1;
                                        double slope = (double) deltaY / deltaX;
                                        if (Math.abs(slope) <= 1) {
                                            for (int x = x1; x <= x2; x++) {
                                                int y = (int) (y1 + slope * (x - x1));
                                                screenBuffer[y * SCREEN_WIDTH + x] = "##";
                                            }
                                        } else {
                                            slope = 1 / slope;
                                            for (int y = y1; y <= y2; y++) {
                                                int x = (int) (x1 + slope * (y - y1));
                                                screenBuffer[y * SCREEN_WIDTH + x] = "##";
                                            }
                                        }
                                    }

                                    static void drawRectangle(int x1, int y1, int x2, int y2) {
                                        drawLine(x1, y1, x1, y2);
                                        drawLine(x1, y1, x2, y1);
                                        drawLine(x2, y1, x2, y2);
                                        drawLine(x1, y2, x2, y2);
                                    }

                                    static void displayScreen() {
                                        System.out.println("┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐");
                                        for (int y = 0; y < SCREEN_HEIGHT; y++) {
                                            System.out.print("│");
                                            for (int x = 0; x < SCREEN_WIDTH; x++) {
                                                System.out.print(screenBuffer[y * SCREEN_WIDTH + x]);
                                            }
                                            System.out.println("│");
                                        }
                                        System.out.println("└──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘");
                                    }
                                }"""
    },
        {
        "ID": 20,
        "Domain": "Encryption",
        "Purpose": "Caesar Cipher",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. CaesarCipher.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 PlainText PIC X(50) VALUE "HELLO".
                        01 EncryptedText PIC X(50).
                        01 Key PIC 9 VALUE 3.
                        01 I PIC 9.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > LENGTH OF PlainText
                                IF PlainText(I) >= 'A' AND PlainText(I) <= 'Z'
                                    COMPUTE EncryptedText(I) = FUNCTION CHAR(UNSTRING FUNCTION UPPER-CASE(PlainText(I)) DELIMITED BY SIZE)
                                    COMPUTE EncryptedText(I) = FUNCTION CHAR(UNSTRING FUNCTION MOD(FUNCTION ORD(EncryptedText(I)) - FUNCTION ORD('A') + Key, 26) + FUNCTION ORD('A'))
                                ELSE
                                    MOVE PlainText(I) TO EncryptedText(I)
                                END-IF
                            END-PERFORM
                            DISPLAY "Encrypted Text: " EncryptedText
                            STOP RUN.""",
        "Java Translation": """public class CaesarCipher {
                                    public static void main(String[] args) {
                                        String plainText = "HELLO";
                                        StringBuilder encryptedText = new StringBuilder();
                                        int key = 3;
                                        for (char c : plainText.toCharArray()) {
                                            if (Character.isLetter(c)) {
                                                char shiftedChar = (char) (((c - 'A' + key) % 26) + 'A');
                                                encryptedText.append(Character.toUpperCase(shiftedChar));
                                            } else {
                                                encryptedText.append(c);
                                            }
                                        }
                                        System.out.println("Encrypted Text: " + encryptedText);
                                    }
                                }"""
    },
    {
        "ID": 21,
        "Domain": "Error Handling",
        "Purpose": "Exception Handling",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. ExceptionHandling.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Number1 PIC 99 VALUE 10.
                        01 Number2 PIC 99 VALUE 0.
                        PROCEDURE DIVISION.
                        P1.
                            DISPLAY "Result: "
                            DIVIDE Number1 BY Number2 GIVING Result
                                ON SIZE ERROR
                                    DISPLAY "Size Error: Cannot Divide by Zero"
                                ON ZERO
                                    DISPLAY "Zero Error: Cannot Divide by Zero"
                            END-DIVIDE
                            STOP RUN.""",
        "Java Translation": """public class ExceptionHandling {
                                    public static void main(String[] args) {
                                        int number1 = 10;
                                        int number2 = 0;
                                        try {
                                            int result = number1 / number2;
                                            System.out.println("Result: " + result);
                                        } catch (ArithmeticException e) {
                                            if (e.getMessage().equals("/ by zero")) {
                                                System.out.println("Zero Error: Cannot Divide by Zero");
                                            } else {
                                                System.out.println("Size Error: " + e.getMessage());
                                            }
                                        }
                                    }
                                }"""
    },
        {
        "ID": 22,
        "Domain": "Data Compression",
        "Purpose": "Run-Length Encoding",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. RunLengthEncoding.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 OriginalString PIC X(100) VALUE "AAABBCCDDEEEE".
                        01 EncodedString PIC X(100).
                        01 Index PIC 99 VALUE 1.
                        01 Count PIC 99.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM UNTIL Index > LENGTH OF OriginalString
                                COMPUTE Count = 1
                                PERFORM UNTIL OriginalString(Index) NOT EQUAL OriginalString(Index + 1) OR Index = LENGTH OF OriginalString
                                    ADD 1 TO Count
                                    ADD 1 TO Index
                                END-PERFORM
                                MOVE OriginalString(Index) TO EncodedString(Index)
                                MOVE Count TO EncodedString(Index + 1)
                                ADD 1 TO Index
                            END-PERFORM
                            DISPLAY "Original String: " OriginalString
                            DISPLAY "Encoded String: " EncodedString
                            STOP RUN.""",
        "Java Translation": """public class RunLengthEncoding {
                                    public static void main(String[] args) {
                                        String originalString = "AAABBCCDDEEEE";
                                        StringBuilder encodedString = new StringBuilder();
                                        int index = 0;
                                        while (index < originalString.length()) {
                                            char currentChar = originalString.charAt(index);
                                            int count = 1;
                                            while (index + count < originalString.length() && originalString.charAt(index + count) == currentChar) {
                                                count++;
                                            }
                                            encodedString.append(currentChar).append(count);
                                            index += count;
                                        }
                                        System.out.println("Original String: " + originalString);
                                        System.out.println("Encoded String: " + encodedString);
                                    }
                                }"""
    },
    {
        "ID": 23,
        "Domain": "Image Processing",
        "Purpose": "Image Resizing",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. ImageResizing.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 OriginalImage PIC X(1000) VALUE "Original Image Data".
                        01 ResizedImage PIC X(500) VALUE SPACE.
                        01 ScaleFactor PIC 99V99 VALUE 0.5.
                        01 OriginalIndex PIC 999 VALUE 1.
                        01 ResizedIndex PIC 999 VALUE 1.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM UNTIL OriginalIndex > LENGTH OF OriginalImage
                                MOVE OriginalImage(OriginalIndex:OriginalIndex + 1) TO ResizedImage(ResizedIndex:ResizedIndex + 1)
                                COMPUTE OriginalIndex = OriginalIndex + 2
                                COMPUTE ResizedIndex = ResizedIndex + 1
                            END-PERFORM
                            DISPLAY "Original Image: " OriginalImage
                            DISPLAY "Resized Image: " ResizedImage
                            STOP RUN.""",
        "Java Translation": """public class ImageResizing {
                                    public static void main(String[] args) {
                                        String originalImage = "Original Image Data";
                                        StringBuilder resizedImage = new StringBuilder();
                                        double scaleFactor = 0.5;
                                        for (int i = 0; i < originalImage.length(); i += 2) {
                                            resizedImage.append(originalImage.charAt(i));
                                        }
                                        System.out.println("Original Image: " + originalImage);
                                        System.out.println("Resized Image: " + resizedImage);
                                    }
                                }"""
    },
    {
        "ID": 24,
        "Domain": "Network Programming",
        "Purpose": "Socket Programming",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. SocketProgramming.
                        ENVIRONMENT DIVISION.
                        CONFIGURATION SECTION.
                        SOURCE-COMPUTER. ALPHABETIC.
                        OBJECT-COMPUTER. ALPHABETIC.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Buffer PIC X(100) VALUE SPACE.
                        01 Port PIC 9999 VALUE 8080.
                        PROCEDURE DIVISION.
                        P1.
                            CALL "INITIALIZE_SOCKET" USING Port
                            CALL "RECEIVE_DATA" USING Buffer
                            DISPLAY "Received Data: " Buffer
                            CALL "CLOSE_SOCKET"
                            STOP RUN.""",
        "Java Translation": """import java.io.BufferedReader;
                                import java.io.InputStreamReader;
                                import java.io.OutputStreamWriter;
                                import java.io.PrintWriter;
                                import java.net.ServerSocket;
                                import java.net.Socket;

                                public class SocketProgramming {
                                    public static void main(String[] args) {
                                        int port = 8080;
                                        try (ServerSocket serverSocket = new ServerSocket(port);
                                             Socket clientSocket = serverSocket.accept();
                                             BufferedReader in = new BufferedReader(new InputStreamReader(clientSocket.getInputStream()));
                                             PrintWriter out = new PrintWriter(new OutputStreamWriter(clientSocket.getOutputStream()))) {
                                            StringBuilder receivedData = new StringBuilder();
                                            String line;
                                            while ((line = in.readLine()) != null) {
                                                receivedData.append(line);
                                            }
                                            System.out.println("Received Data: " + receivedData);
                                        } catch (Exception e) {
                                            e.printStackTrace();
                                        }
                                    }
                                }"""
    },
    {
        "ID": 25,
        "Domain": "Natural Language Processing",
        "Purpose": "Text Tokenization",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. TextTokenization.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Text PIC X(100) VALUE "This is a sample text.".
                        01 Token OCCURS 10 TIMES PIC X(10).
                        01 Delimiters PIC X(10) VALUE " ".
                        01 TokenCount PIC 99 VALUE 1.
                        PROCEDURE DIVISION.
                        P1.
                            UNSTRING Text DELIMITED BY Delimiters INTO Token(TokenCount)
                            PERFORM VARYING TokenCount FROM 1 BY 1 UNTIL Token(TokenCount) = SPACE
                                DISPLAY "Token " TokenCount ": " Token(TokenCount)
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class TextTokenization {
                                    public static void main(String[] args) {
                                        String text = "This is a sample text.";
                                        String[] tokens = text.split(" ");
                                        int tokenCount = 1;
                                        for (String token : tokens) {
                                            System.out.println("Token " + tokenCount + ": " + token);
                                            tokenCount++;
                                        }
                                    }
                                }"""
    },
    {
        "ID": 26,
        "Domain": "Web Development",
        "Purpose": "REST API Development",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. RESTAPI.
                        ENVIRONMENT DIVISION.
                        CONFIGURATION SECTION.
                        SOURCE-COMPUTER. ALPHABETIC.
                        OBJECT-COMPUTER. ALPHABETIC.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Buffer PIC X(100) VALUE SPACE.
                        01 Port PIC 9999 VALUE 8080.
                        PROCEDURE DIVISION.
                        P1.
                            CALL "INITIALIZE_SERVER" USING Port
                            CALL "LISTEN_FOR_REQUESTS"
                            CALL "SEND_RESPONSE" USING Buffer
                            DISPLAY "Response Sent: " Buffer
                            CALL "CLOSE_SERVER"
                            STOP RUN.""",
        "Java Translation": """import java.io.BufferedReader;
                                import java.io.InputStreamReader;
                                import java.io.OutputStreamWriter;
                                import java.io.PrintWriter;
                                import java.net.ServerSocket;
                                import java.net.Socket;

                                public class RESTAPI {
                                    public static void main(String[] args) {
                                        int port = 8080;
                                        try (ServerSocket serverSocket = new ServerSocket(port);
                                             Socket clientSocket = serverSocket.accept();
                                             BufferedReader in = new BufferedReader(new InputStreamReader(clientSocket.getInputStream()));
                                             PrintWriter out = new PrintWriter(new OutputStreamWriter(clientSocket.getOutputStream()))) {
                                            String request = in.readLine();
                                            String response = "Response to " + request;
                                            out.println(response);
                                            out.flush();
                                            System.out.println("Response Sent: " + response);
                                        } catch (Exception e) {
                                            e.printStackTrace();
                                        }
                                    }
                                }"""
    },
    {
        "ID": 27,
        "Domain": "Robotics",
        "Purpose": "Path Planning",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. PathPlanning.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Grid OCCURS 5 TIMES PIC X(5) VALUE '00000',
                                                        '11100',
                                                        '00100',
                                                        '00111',
                                                        '11110'.
                        01 StartRow PIC 99 VALUE 1.
                        01 StartCol PIC 99 VALUE 1.
                        01 GoalRow PIC 99 VALUE 5.
                        01 GoalCol PIC 99 VALUE 5.
                        01 PathFound PIC X VALUE 'N'.
                        PROCEDURE DIVISION.
                        P1.
                            MOVE StartRow TO CurrentRow
                            MOVE StartCol TO CurrentCol
                            PERFORM UNTIL (CurrentRow = GoalRow AND CurrentCol = GoalCol) OR PathFound = 'Y'
                                MOVE Grid(CurrentRow:CurrentCol) TO CurrentCell
                                IF CurrentRow < GoalRow AND Grid(CurrentRow + 1:CurrentCol) = '0'
                                    MOVE CurrentRow + 1 TO CurrentRow
                                ELSE IF CurrentCol < GoalCol AND Grid(CurrentRow:CurrentCol + 1) = '0'
                                    MOVE CurrentCol + 1 TO CurrentCol
                                ELSE
                                    MOVE 'Y' TO PathFound
                                END-IF
                            END-PERFORM
                            IF PathFound = 'Y'
                                DISPLAY "Path Found"
                            ELSE
                                DISPLAY "Path Not Found"
                            END-IF
                            STOP RUN.""",
        "Java Translation": """public class PathPlanning {
                                    public static void main(String[] args) {
                                        char[][] grid = {
                                            {'0', '0', '0', '0', '0'},
                                            {'1', '1', '1', '0', '0'},
                                            {'0', '0', '1', '0', '0'},
                                            {'0', '0', '1', '1', '1'},
                                            {'1', '1', '1', '1', '0'}
                                        };
                                        int startRow = 0;
                                        int startCol = 0;
                                        int goalRow = 4;
                                        int goalCol = 4;
                                        int currentRow = startRow;
                                        int currentCol = startCol;
                                        boolean pathFound = false;
                                        while ((currentRow != goalRow || currentCol != goalCol) && !pathFound) {
                                            if (currentRow < goalRow && grid[currentRow + 1][currentCol] == '0') {
                                                currentRow++;
                                            } else if (currentCol < goalCol && grid[currentRow][currentCol + 1] == '0') {
                                                currentCol++;
                                            } else {
                                                pathFound = true;
                                            }
                                        }
                                        if (pathFound) {
                                            System.out.println("Path Found");
                                        } else {
                                            System.out.println("Path Not Found");
                                        }
                                    }
                                }"""
    },
    {
        "ID": 28,
        "Domain": "Financial Analysis",
        "Purpose": "Stock Market Analysis",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. StockMarketAnalysis.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 StockPrices OCCURS 10 TIMES PIC 999V99 VALUE 100.00, 110.50, 95.75, 105.25, 112.75, 98.50, 120.00, 115.75, 125.50, 130.25.
                        01 NumPrices PIC 99 VALUE 10.
                        01 TotalPrice PIC 9999V99.
                        01 AveragePrice PIC 999V99.
                        01 MaxPrice PIC 999V99 VALUE 0.
                        01 MinPrice PIC 999V99 VALUE 999.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > NumPrices
                                COMPUTE TotalPrice = TotalPrice + StockPrices(I)
                                IF StockPrices(I) > MaxPrice
                                    MOVE StockPrices(I) TO MaxPrice
                                END-IF
                                IF StockPrices(I) < MinPrice
                                    MOVE StockPrices(I) TO MinPrice
                                END-IF
                            END-PERFORM
                            COMPUTE AveragePrice = TotalPrice / NumPrices
                            DISPLAY "Total Price: " TotalPrice
                            DISPLAY "Average Price: " AveragePrice
                            DISPLAY "Max Price: " MaxPrice
                            DISPLAY "Min Price: " MinPrice
                            STOP RUN.""",
        "Java Translation": """public class StockMarketAnalysis {
                                    public static void main(String[] args) {
                                        double[] stockPrices = {100.00, 110.50, 95.75, 105.25, 112.75, 98.50, 120.00, 115.75, 125.50, 130.25};
                                        int numPrices = 10;
                                        double totalPrice = 0;
                                        double maxPrice = 0;
                                        double minPrice = 999;
                                        for (int i = 0; i < numPrices; i++) {
                                            totalPrice += stockPrices[i];
                                            if (stockPrices[i] > maxPrice) {
                                                maxPrice = stockPrices[i];
                                            }
                                            if (stockPrices[i] < minPrice) {
                                                minPrice = stockPrices[i];
                                            }
                                        }
                                        double averagePrice = totalPrice / numPrices;
                                        System.out.println("Total Price: " + totalPrice);
                                        System.out.println("Average Price: " + averagePrice);
                                        System.out.println("Max Price: " + maxPrice);
                                        System.out.println("Min Price: " + minPrice);
                                    }
                                }"""
    },
    {
        "ID": 29,
        "Domain": "Graph Theory",
        "Purpose": "Shortest Path",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. ShortestPath.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 Graph OCCURS 5 TIMES PIC X(5) VALUE '00000',
                                                            '11100',
                                                            '00100',
                                                            '00111',
                                                            '11110'.
                        01 StartNode PIC 99 VALUE 1.
                        01 GoalNode PIC 99 VALUE 5.
                        01 VisitedNodes OCCURS 5 TIMES PIC X VALUE 'N'.
                        01 Queue OCCURS 10 TIMES PIC 99.
                        01 Front PIC 99 VALUE 1.
                        01 Rear PIC 99 VALUE 1.
                        01 PathFound PIC X VALUE 'N'.
                        PROCEDURE DIVISION.
                        P1.
                            MOVE StartNode TO Queue(Rear)
                            MOVE 'Y' TO VisitedNodes(StartNode)
                            PERFORM UNTIL Front > Rear OR PathFound = 'Y'
                                IF Queue(Front) = GoalNode
                                    MOVE 'Y' TO PathFound
                                ELSE
                                    PERFORM VARYING I FROM 1 BY 1 UNTIL I > 5
                                        IF Graph(Queue(Front):I) = '1' AND VisitedNodes(I) = 'N'
                                            COMPUTE Rear = Rear + 1
                                            MOVE I TO Queue(Rear)
                                            MOVE 'Y' TO VisitedNodes(I)
                                        END-IF
                                    END-PERFORM
                                    COMPUTE Front = Front + 1
                                END-IF
                            END-PERFORM
                            IF PathFound = 'Y'
                                DISPLAY "Shortest Path Found"
                            ELSE
                                DISPLAY "Shortest Path Not Found"
                            END-IF
                            STOP RUN.""",
        "Java Translation": """public class ShortestPath {
                                    public static void main(String[] args) {
                                        char[][] graph = {
                                            {'0', '0', '0', '0', '0'},
                                            {'1', '1', '1', '0', '0'},
                                            {'0', '0', '1', '0', '0'},
                                            {'0', '0', '1', '1', '1'},
                                            {'1', '1', '1', '1', '0'}
                                        };
                                        int startNode = 1;
                                        int goalNode = 5;
                                        boolean[] visitedNodes = new boolean[5];
                                        int[] queue = new int[10];
                                        int front = 0;
                                        int rear = 0;
                                        boolean pathFound = false;
                                        queue[rear] = startNode;
                                        rear++;
                                        visitedNodes[startNode] = true;
                                        while (front < rear && !pathFound) {
                                            if (queue[front] == goalNode) {
                                                pathFound = true;
                                            } else {
                                                for (int i = 0; i < 5; i++) {
                                                    if (graph[queue[front]][i] == '1' && !visitedNodes[i]) {
                                                        queue[rear] = i;
                                                        rear++;
                                                        visitedNodes[i] = true;
                                                    }
                                                }
                                                front++;
                                            }
                                        }
                                        if (pathFound) {
                                            System.out.println("Shortest Path Found");
                                        } else {
                                            System.out.println("Shortest Path Not Found");
                                        }
                                    }
                                }"""
    },
    {
        "ID": 30,
        "Domain": "Distributed Systems",
        "Purpose": "MapReduce",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. MapReduce.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 InputData PIC X(100) VALUE "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.".
                        01 OutputData PIC X(100) VALUE SPACE.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM UNTIL InputData = SPACE
                                UNSTRING InputData DELIMITED BY SPACE INTO OutputData
                                DISPLAY "Word: " OutputData
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class MapReduce {
                                    public static void main(String[] args) {
                                        String inputData = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua.";
                                        String[] words = inputData.split(" ");
                                        for (String word : words) {
                                            System.out.println("Word: " + word);
                                        }
                                    }
                                }"""
    },
    {
        "ID": 31,
        "Domain": "Blockchain",
        "Purpose": "Cryptocurrency Implementation",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. Cryptocurrency.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 TransactionHistory OCCURS 10 TIMES.
                            05 Sender PIC X(10) VALUE "Alice".
                            05 Receiver PIC X(10) VALUE "Bob".
                            05 Amount PIC 999V99 VALUE 10.50.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > 10
                                DISPLAY "Sender: " TransactionHistory(Sender)
                                DISPLAY "Receiver: " TransactionHistory(Receiver)
                                DISPLAY "Amount: " TransactionHistory(Amount)
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class Cryptocurrency {
                                    public static void main(String[] args) {
                                        String[] sender = {"Alice", "Alice", "Alice", "Alice", "Alice", "Alice", "Alice", "Alice", "Alice", "Alice"};
                                        String[] receiver = {"Bob", "Bob", "Bob", "Bob", "Bob", "Bob", "Bob", "Bob", "Bob", "Bob"};
                                        double[] amount = {10.50, 10.50, 10.50, 10.50, 10.50, 10.50, 10.50, 10.50, 10.50, 10.50};
                                        for (int i = 0; i < 10; i++) {
                                            System.out.println("Sender: " + sender[i]);
                                            System.out.println("Receiver: " + receiver[i]);
                                            System.out.println("Amount: " + amount[i]);
                                        }
                                    }
                                }"""
    },
    {
        "ID": 32,
        "Domain": "Augmented Reality",
        "Purpose": "Object Detection",
        "COBOL Code": """IDENTIFICATION DIVISION.
                        PROGRAM-ID. ObjectDetection.
                        DATA DIVISION.
                        WORKING-STORAGE SECTION.
                        01 CameraFrame PIC X(1000) VALUE "Camera frame data".
                        01 DetectedObjects OCCURS 10 TIMES.
                            05 ObjectName PIC X(10).
                            05 ObjectLocationX PIC 999.
                            05 ObjectLocationY PIC 999.
                        PROCEDURE DIVISION.
                        P1.
                            PERFORM VARYING I FROM 1 BY 1 UNTIL I > 10
                                DISPLAY "Object Name: " DetectedObjects(ObjectName)
                                DISPLAY "Object Location X: " DetectedObjects(ObjectLocationX)
                                DISPLAY "Object Location Y: " DetectedObjects(ObjectLocationY)
                            END-PERFORM
                            STOP RUN.""",
        "Java Translation": """public class ObjectDetection {
                                    public static void main(String[] args) {
                                        String cameraFrame = "Camera frame data";
                                        String[] objectName = new String[10];
                                        int[] objectLocationX = new int[10];
                                        int[] objectLocationY = new int[10];
                                        for (int i = 0; i < 10; i++) {
                                            System.out.println("Object Name: " + objectName[i]);
                                            System.out.println("Object Location X: " + objectLocationX[i]);
                                            System.out.println("Object Location Y: " + objectLocationY[i]);
                                        }
                                    }
                                }"""
    }
]

# file path for the CSV file
csv_file = "cobol_java_pairs.csv"

# Write to the CSV file
with open(csv_file, "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["ID", "Domain", "Purpose", "COBOL Code", "Java Translation"])
    writer.writeheader()
    for pair in cobol_java_pairs:
        writer.writerow(pair)

print(f"CSV file '{csv_file}' has been created successfully.")

CSV file 'cobol_java_pairs.csv' has been created successfully.


Reading the CSV file

In [14]:
import pandas as pd
df=pd.read_csv("/content/cobol_java_pairs_updated.csv")
df

,ID,Domain,Purpose,COBOL Code,Java Translation
0,1,General,HelloWorld,IDENTIFICATION DIVISION.\n ...,public class HelloWorld {\n ...
1,2,Arithmetic,AddNumbers,IDENTIFICATION DIVISION.\n ...,public class AddNumbers {\n ...
2,3,Arithmetic,LargestNumber,IDENTIFICATION DIVISION.\n ...,public class LargestNumber {\n ...
3,4,Mathematical,Factorial,IDENTIFICATION DIVISION.\n ...,public class Factorial {\n ...
4,5,Mathematical,FibonacciSeries,IDENTIFICATION DIVISION.\n ...,public class FibonacciSeries {\n ...
...,...,...,...,...,...
68,69,Advanced,KruskalAlgorithm,IDENTIFICATION DIVISION.\n ...,public class KruskalAlgorithm {\n ...
69,70,Advanced,DijkstraAlgorithm,IDENTIFICATION DIVISION.\n ...,public class DijkstraAlgorithm {\n ...
70,71,Advanced,DepthFirstSearchAlgorithm,IDENTIFICATION DIVISION.\n ...,public class DepthFirstSearchAlgorithm {\n ...
71,72,Advanced,BreadthFirstSearchAlgorithm,IDENTIFICATION DIVISION.\n ...,public class BreadthFirstSearchAlgorithm {\n ...


# **Pre-processing the DATA of COBOL and JAVA code**

Pre-processing code defined inside the functions for COBOL and JAVA code

In [15]:
import re

# Normalize the tokens (e.g., lowercase conversion)
def tokenize_code(code):
    tokens = re.findall(r"[\w']+|[.,!?;]", code)
    return tokens

# Normalize the tokens (lowercase conversion)
def normalize_code(tokens):
    normalized_tokens = [token.lower() for token in tokens]
    return normalized_tokens

Structures the preprocessed data into a list of dictionaries

In [16]:
def preprocess_dataset(dataset):
    preprocessed_dataset = []
    for pair in dataset:
        preprocessed_pair = {}
        preprocessed_pair["ID"] = pair["ID"]
        preprocessed_pair["Domain"] = pair["Domain"]
        preprocessed_pair["Purpose"] = pair["Purpose"]

        # Preprocess COBOL code
        cobol_code = pair["COBOL Code"]
        cobol_tokens = tokenize_code(cobol_code)
        cobol_normalized_tokens = normalize_code(cobol_tokens)
        preprocessed_pair["Preprocessed COBOL"] = cobol_normalized_tokens

        # Preprocess Java code
        java_code = pair["Java Translation"]
        java_tokens = tokenize_code(java_code)
        java_normalized_tokens = normalize_code(java_tokens)
        preprocessed_pair["Preprocessed Java"] = java_normalized_tokens

        preprocessed_dataset.append(preprocessed_pair)

    return preprocessed_dataset

preprocessed_dataset = preprocess_dataset(cobol_java_pairs)
print("Preprocessed dataset:", preprocessed_dataset)


Preprocessed dataset: [{'ID': 1, 'Domain': 'General', 'Purpose': 'HelloWorld', 'Preprocessed COBOL': ['identification', 'division', '.', 'program', 'id', '.', 'helloworld', '.', 'procedure', 'division', '.', 'p1', '.', 'display', 'hello', ',', 'world', '!', '.', 'stop', 'run', '.'], 'Preprocessed Java': ['public', 'class', 'helloworld', 'public', 'static', 'void', 'main', 'string', 'args', 'system', '.', 'out', '.', 'println', 'hello', ',', 'world', '!', ';']}, {'ID': 2, 'Domain': 'Arithmetic', 'Purpose': 'AddNumbers', 'Preprocessed COBOL': ['identification', 'division', '.', 'program', 'id', '.', 'addnumbers', '.', 'data', 'division', '.', 'working', 'storage', 'section', '.', '01', 'a', 'pic', '9', '2', 'value', '10', '.', '01', 'b', 'pic', '9', '2', 'value', '20', '.', '01', 'sum', 'pic', '9', '4', '.', 'procedure', 'division', '.', 'p1', '.', 'compute', 'sum', 'a', 'b', '.', 'display', 'sum', ',', 'sum', '.', 'stop', 'run', '.'], 'Preprocessed Java': ['public', 'class', 'addnumbers

# **Creating Pre-processed CSV file**

In [17]:
csv_file_preprocessed = "cobol_java_pairs_preprocessed.csv"

# Write to the CSV file
with open(csv_file_preprocessed, "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=["ID", "Domain", "Purpose", "Preprocessed COBOL", "Preprocessed Java"])
    writer.writeheader()
    for pair in preprocessed_dataset:
        writer.writerow(pair)

print(f"Preprocessed CSV file '{csv_file_preprocessed}' has been created successfully.")

Preprocessed CSV file 'cobol_java_pairs_preprocessed.csv' has been created successfully.


Reading the Pre-processed CSV file

In [18]:
import pandas as pd
preprocessed_dataset=pd.read_csv("/content/cobol_java_pairs_preprocessed.csv")
preprocessed_dataset

,ID,Domain,Purpose,Preprocessed COBOL,Preprocessed Java
0,1,General,HelloWorld,"['identification', 'division', '.', 'program',...","['public', 'class', 'helloworld', 'public', 's..."
1,2,Arithmetic,AddNumbers,"['identification', 'division', '.', 'program',...","['public', 'class', 'addnumbers', 'public', 's..."
2,3,Arithmetic,LargestNumber,"['identification', 'division', '.', 'program',...","['public', 'class', 'largestnumber', 'public',..."
3,4,Mathematical,Factorial,"['identification', 'division', '.', 'program',...","['public', 'class', 'factorial', 'public', 'st..."
4,5,Mathematical,FibonacciSeries,"['identification', 'division', '.', 'program',...","['public', 'class', 'fibonacciseries', 'public..."
5,6,File Handling,ReadFile,"['identification', 'division', '.', 'program',...","['import', 'java', '.', 'io', '.', 'bufferedre..."
6,7,String Manipulation,ReverseString,"['identification', 'division', '.', 'program',...","['public', 'class', 'reversestring', 'public',..."
7,8,Sorting,BubbleSort,"['identification', 'division', '.', 'program',...","['public', 'class', 'bubblesort', 'public', 's..."
8,9,Data Structures,Linked List,"['identification', 'division', '.', 'program',...","['public', 'class', 'linkedlist', 'static', 'c..."
9,10,Networking,TCP Client,"['identification', 'division', '.', 'program',...","['import', 'java', '.', 'io', '.', ';', 'impor..."


# **Feature Extraction, Training Classifier Prediction and Evaluation**

In [19]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, classification_report

Extract features (token frequencies) from preprocessed data

In [20]:
cobol_tokens = preprocessed_dataset["Preprocessed COBOL"].apply(lambda x: x.split())
java_tokens = preprocessed_dataset["Preprocessed Java"].apply(lambda x: x.split())
cobol_token_strings = [' '.join(tokens) for tokens in cobol_tokens]
java_token_strings = [' '.join(tokens) for tokens in java_tokens]

Use CountVectorizer for token frequencies

In [21]:
vectorizer = CountVectorizer()
X_cobol = vectorizer.fit_transform(cobol_token_strings)
X_java = vectorizer.transform(java_token_strings)

Convert X_cobol to dense array

In [22]:
X_cobol_dense = X_cobol.toarray()

# **Building decision tree classifier model**

In [23]:
classifier = DecisionTreeClassifier()
classifier.fit(X_cobol_dense, preprocessed_dataset["Preprocessed Java"])  # Use the raw Java code strings as labels


DecisionTreeClassifier()

Predict Java code tokens based on COBOL code tokens

In [24]:
predictions = classifier.predict(X_cobol_dense)

Evaluate the classifier

In [25]:
accuracy = accuracy_score(preprocessed_dataset["Preprocessed Java"], predictions)
print("Accuracy:", accuracy)

Accuracy: 1.0


Print classification report for more detailed evaluation

In [26]:
print("Classification Report:")
print(classification_report(preprocessed_dataset["Preprocessed Java"], predictions))


Classification Report:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

# **k-fold cross-validation for hyperparameter tuning using grid search with a decision tree classifier.**

In [36]:
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV

# Assuming "preprocessed_dataset" contains your preprocessed data
y_java_over = preprocessed_dataset["Preprocessed Java"]
X_cobol_over = preprocessed_dataset["Preprocessed COBOL"]

# Determine the number of splits
n_splits = min(2, len(np.unique(y_java_over)))

# Initialize KFold with the adjusted number of splits
kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform COBOL and Java token strings
X_cobol_tfidf = tfidf_vectorizer.fit_transform(X_cobol_over)
X_java_tfidf = tfidf_vectorizer.transform(y_java_over)

# Perform grid search using k-fold cross-validation
grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, cv=kf)
grid_search.fit(X_cobol_tfidf, y_java_over)

print("Best Hyperparameters:", grid_search.best_params_)

best_classifier = grid_search.best_estimator_
best_classifier.fit(X_cobol_tfidf, y_java_over)


Best Hyperparameters: {'criterion': 'gini', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}


DecisionTreeClassifier()

In [37]:
misclassified_samples = preprocessed_dataset[preprocessed_dataset["Preprocessed Java"] != predictions]
print("Misclassified Samples:")
print(misclassified_samples)


Misclassified Samples:
Empty DataFrame
Columns: [ID, Domain, Purpose, Preprocessed COBOL, Preprocessed Java]
Index: []


# **Random Forest Model Building**

In [38]:
from sklearn.ensemble import RandomForestClassifier

random_forest_classifier = RandomForestClassifier()
random_forest_classifier.fit(X_cobol_dense, preprocessed_dataset["Preprocessed Java"])


RandomForestClassifier()

In [39]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

predictions = random_forest_classifier.predict(X_cobol_dense)

accuracy = accuracy_score(preprocessed_dataset["Preprocessed Java"], predictions)
print("Accuracy:", accuracy)

print("Classification Report:")
print(classification_report(preprocessed_dataset["Preprocessed Java"], predictions))

print("Confusion Matrix:")
print(confusion_matrix(preprocessed_dataset["Preprocessed Java"], predictions))


Accuracy: 1.0
Classification Report:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   

In [43]:
import re

def generate_java_code_from_cobol(cobol_code, vectorizer, classifier):
    cobol_tokens = tokenize_code(cobol_code)
    cobol_normalized_tokens = normalize_code(cobol_tokens)
    cobol_token_string = ' '.join(cobol_normalized_tokens)

    X_cobol = vectorizer.transform([cobol_token_string])
    X_cobol_dense = X_cobol.toarray()

    java_code_tokens = classifier.predict(X_cobol_dense)

    java_code = convert_tokens_to_syntactic(java_code_tokens)

    return format_java_code(java_code)

def convert_tokens_to_syntactic(java_code_tokens):
    syntactic_java_code = ""
    for token in java_code_tokens:
        if token.lower() == "identification":
            continue  # Skip COBOL identification division
        elif token.lower() == "division" or token == ".":
            syntactic_java_code += ";\n"  # End of statement in Java
        elif token.lower() == "program-id":
            syntactic_java_code += "public class "  # Start of Java class
        elif token.lower() == "procedure":
            syntactic_java_code += "public static void main(String[] args) {\n"  # Start of main method
        elif token.lower() == "display":
            syntactic_java_code += "System.out.println"  # Equivalent Java print statement
        elif token == '"':
            syntactic_java_code += "("  # Start of string in Java print statement
        elif token == '.':
            syntactic_java_code += ");"  # End of string in Java print statement
        elif token.isdigit():
            syntactic_java_code += token  # Numbers remain unchanged
        else:
            syntactic_java_code += token.lower()  # Convert to lowercase
    syntactic_java_code += "}\n"  # End of class
    return syntactic_java_code

def format_java_code(java_code):
    # Add indentation
    formatted_java_code = ""
    indentation_level = 0
    for line in java_code.split('\n'):
        if "{" in line:
            formatted_java_code += "\t" * indentation_level + line + "\n"
            indentation_level += 1
        elif "}" in line:
            indentation_level -= 1
            formatted_java_code += "\t" * indentation_level + line + "\n"
        else:
            formatted_java_code += "\t" * indentation_level + line + "\n"
    return formatted_java_code

def tokenize_code(code):
    # Tokenize the code based on whitespace and special characters
    tokens = re.findall(r'\w+|[^\w\s]', code)
    return tokens

def normalize_code(tokens):
    # Normalize the tokens by converting them to lowercase
    normalized_tokens = [token.lower() for token in tokens]
    return normalized_tokens


cobol_code = input("Enter COBOL code: ")

java_code = generate_java_code_from_cobol(cobol_code, vectorizer, random_forest_classifier)

print("Generated Java code:")
print(java_code)


Enter COBOL code: IDENTIFICATION DIVISION. PROGRAM-ID. SUPPLY-CHAIN-SYSTEM.  DATA DIVISION. WORKING-STORAGE SECTION. 01 PRODUCT-RECORD.    05 PRODUCT-ID         PIC X(10).    05 PRODUCT-NAME       PIC X(30).    05 PRODUCT-QUANTITY   PIC 9(8).    05 PRODUCT-PRICE      PIC 9(6)V99.  01 WAREHOUSE-RECORD.    05 WAREHOUSE-ID       PIC X(10).    05 WAREHOUSE-NAME     PIC X(30).    05 WAREHOUSE-LOCATION PIC X(50).  01 ORDER-RECORD.    05 ORDER-ID           PIC X(10).    05 PRODUCT-ID         PIC X(10).    05 ORDER-QUANTITY     PIC 9(8).    05 ORDER-DATE         PIC 9(8).  01 PRODUCT-TABLE.    05 PRODUCT-ENTRY OCCURS 100 TIMES.       10 PRODUCT-ID         PIC X(10).       10 PRODUCT-NAME       PIC X(30).       10 PRODUCT-QUANTITY   PIC 9(8).       10 PRODUCT-PRICE      PIC 9(6)V99.  01 WAREHOUSE-TABLE.    05 WAREHOUSE-ENTRY OCCURS 10 TIMES.       10 WAREHOUSE-ID       PIC X(10).       10 WAREHOUSE-NAME     PIC X(30).       10 WAREHOUSE-LOCATION PIC X(50).  01 ORDER-TABLE.    05 ORDER-ENTRY OCC

In [ ]:
from sklearn.ensemble import RandomForestClassifier

def generate_java_code_from_cobol(cobol_code, vectorizer, classifier):

    cobol_tokens = tokenize_code(cobol_code)
    cobol_normalized_tokens = normalize_code(cobol_tokens)
    cobol_token_string = ' '.join(cobol_normalized_tokens)

    X_cobol = vectorizer.transform([cobol_token_string])
    X_cobol_dense = X_cobol.toarray()

    java_code_tokens = classifier.predict(X_cobol_dense)

    java_code = ' '.join(java_code_tokens)

    return java_code

cobol_code = input("Enter COBOL code: ")

java_code = generate_java_code_from_cobol(cobol_code, vectorizer, random_forest_classifier)

print("Generated Java code:")
print(java_code)


In [ ]:
# Filter misclassified samples
misclassified_samples = preprocessed_dataset[preprocessed_dataset["Preprocessed Java"] != predictions]

# Check if there are any misclassified samples
if misclassified_samples.empty:
    print("No misclassified samples found.")
else:
    # Print a subset of misclassified samples for analysis
    num_samples_to_print = min(5, len(misclassified_samples))  # Print up to 5 samples
    print("Misclassified Samples:")
    for i in range(num_samples_to_print):
        print(f"Sample {i + 1}:")
        print("COBOL Code:", misclassified_samples.iloc[i]["COBOL Code"])
        print("Predicted Java Code:", predictions[misclassified_samples.index[i]])
        print("True Java Code:", misclassified_samples.iloc[i]["Preprocessed Java"])
        print()



No misclassified samples found.


In [ ]:
import re
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline

# Define functions for tokenization and normalization
def tokenize_code(code):
    # Tokenize the code by splitting on whitespace and punctuation
    tokens = re.findall(r"[\w']+|[.,!?;]", code)
    return tokens

def normalize_code(tokens):
    # Normalize the tokens (e.g., lowercase conversion)
    normalized_tokens = [token.lower() for token in tokens]
    return normalized_tokens

def preprocess_code(code):
    # Tokenize and normalize the code
    tokens = tokenize_code(code)
    normalized_tokens = normalize_code(tokens)
    return normalized_tokens

# Load preprocessed dataset (if available)
preprocessed_dataset = [
    {
        "ID": 1,
        "Domain": "General",
        "Purpose": "HelloWorld",
        "Preprocessed COBOL": ["identification", "division", "program-id", "helloworld", "procedure", "division", "p1", "display", "hello", "world", "stop", "run"],
        "Preprocessed Java": ["public", "class", "helloworld", "public", "static", "void", "main", "string", "args", "system", "out", "println", "hello", "world"]
    },
    # Add more preprocessed data here
]

# Extract features (token frequencies) from preprocessed data
cobol_tokens = [pair["Preprocessed COBOL"] for pair in preprocessed_dataset]
java_tokens = [pair["Preprocessed Java"] for pair in preprocessed_dataset]
cobol_token_strings = [' '.join(tokens) for tokens in cobol_tokens]
java_token_strings = [' '.join(tokens) for tokens in java_tokens]

# Use CountVectorizer for token frequencies
vectorizer = CountVectorizer()
X_cobol = vectorizer.fit_transform(cobol_token_strings + java_token_strings).toarray()

# Train a simple decision tree classifier
classifier = DecisionTreeClassifier()
classifier.fit(X_cobol[:len(cobol_token_strings)], X_cobol[len(cobol_token_strings):])

# Define the conversion function
def convert_cobol_to_java(cobol_code):
    cobol_tokens = preprocess_code(cobol_code)
    cobol_token_string = ' '.join(cobol_tokens)
    cobol_vector = vectorizer.transform([cobol_token_string]).toarray()
    java_vector = classifier.predict(cobol_vector)
    java_tokens = vectorizer.inverse_transform(java_vector)[0]
    java_code = ' '.join(java_tokens)
    return java_code

# Define a function to handle user input
def main():
    print("Welcome to COBOL to Java converter!")
    while True:
        cobol_code = input("Enter COBOL code (or 'exit' to quit): ")
        if cobol_code.lower() == 'exit':
            print("Exiting...")
            break
        try:
            java_code = convert_cobol_to_java(cobol_code)
            print("Converted Java code:")
            print(java_code)
        except Exception as e:
            print("Error:", e)

if __name__ == "__main__":
    main()


Welcome to COBOL to Java converter!
Enter COBOL code (or 'exit' to quit): IDENTIFICATION DIVISION. PROGRAM-ID. AddTwoNumbers.  DATA DIVISION. WORKING-STORAGE SECTION.    01 Num1       PIC 9(5).    01 Num2       PIC 9(5).    01 Sum        PIC 9(6).     PROCEDURE DIVISION.    DISPLAY "Enter the first number: ".    ACCEPT Num1.    DISPLAY "Enter the second number: ".    ACCEPT Num2.        COMPUTE Sum = Num1 + Num2.        DISPLAY "The sum of " Num1 " and " Num2 " is " Sum "."        STOP RUN.
Converted Java code:
args class hello helloworld main out println public static string system void world


KeyboardInterrupt: Interrupted by user